1) En Mercadodesregulado se registran en un RDD todas las ventas (id_producto,o,
id_comprador, id_vendedor, pais, tiene_reclamo, año, mes, precio_local) y uno de
productos (id_producto, titulo, categoria).
Mercadodesregulado tiene una muy buena relación con el gobierno argentino y el INDEC
pidió su ayuda para medir la inflación entre diciembre de 2018 y diciembre de 2019. Para
esto se quiere conocer:

a) El precio promedio para cada producto de categoría “smartphone” vendido en
diciembre de 2018 y diciembre de 2019 en argentina (10 ptos)

b) El promedio de la relación entre el precio de los productos (siempre para la categoría
smartphone) que se vendieron en diciembre de 2018 y después en diciembre de 2019 en
argentina. La relación puede calcularse como promedio_dic_2019 / promedio_dic_2018
para cada uno de los product_ids. Luego se pide el promedio de las relaciones (20 ptos)
Un ejemplo de ambos puntos para los siguientes datos:

[(1,10,25,"AR",0,2018,"Diciembre",10000),
(1,45,27,"AR",0,2018,"Diciembre",11000),
(1,21,25,"AR",0,2019,"Diciembre",25000),
(2,78,26,"AR",1,2018,"Diciembre",30000),
(2,31,26,"AR",1,2019,"Diciembre",50000),
(3,103,14,"AR",0,2018,"Diciembre",40000),
(3,542,54,"BR",0,2019,"Diciembre",42000)]

Para el punto a)
Promedio diciembre 2018:
(1, 10500)
(2, 30000)
(3, 40000)
Promedio diciembre 2019:
(1, 25000)
(2, 50000)
Para el punto b)
((25000/10500)+(50000/30000))/2 = 2.02...
Resolver los puntos usando la API de RDDs de PySpark. (30 pts)

In [37]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

salesRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2020-1C-3OpSales.csv', header=True).rdd
productsRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2020-1C-3OpProducts.csv', header=True).rdd

salesRdd = salesRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), pd.to_numeric(x[1], 'coerce')
                                   , pd.to_numeric(x[2], 'coerce'), x[3], x[4],
                                   pd.to_numeric(x[5], 'coerce'), pd.to_numeric(x[6], 'coerce')
                                   , pd.to_numeric(x[7], 'coerce')))

productsRdd = productsRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1], x[2]))

smartphonesId = productsRdd.filter(lambda x: x[2] == 'Smartphone').map(lambda x: (x[0], 1))

pricesInDecPerSmartphone = salesRdd.filter(lambda x: (x[6] == 12) & ((x[5] == 2018) | (x[5] == 2019)) & (x[3] == 'Argentina'))\
        .map(lambda x: (x[0], (x[5], x[7]))).join(smartphonesId).map(lambda x: ((x[0], x[1][0][0]), (x[1][0][1], x[1][1])))\
         .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).mapValues(lambda x: x[0]/x[1]).cache()

pricesInDecPerSmartphone.collect()

[((2, 2018), 581.3333333333334),
 ((10, 2018), 484.5),
 ((3, 2019), 371.0),
 ((7, 2019), 923.0),
 ((2, 2019), 967.0),
 ((10, 2019), 219.0),
 ((3, 2018), 590.0),
 ((7, 2018), 370.5),
 ((1, 2018), 496.0)]

In [38]:
relationPrice = pricesInDecPerSmartphone.map(lambda x: (x[0][0], (None if (x[0][1] == 2019) else x[1]
                                                    , None if (x[0][1] == 2018) else x[1]))) \
        .reduceByKey(lambda x,y: (x[0] if (y[0] is None) else y[0], x[1] if (y[1] is None) else y[1])) \
        .filter(lambda x: (x[1][0] is not None) & (x[1][1] is not None)).map(lambda x: (x[1][1]/x[1][0], 1)) \
        .reduce(lambda x,y: (x[0] + y[0], x[1] + y[1]))

relationPrice[0] / relationPrice[1]

1.3088678611477653